In [ ]:
import sqlalchemy
from sqlalchemy import create_engine, Column, String, Integer, Numeric, MetaData, Table, ForeignKey
from sqlalchemy.orm import create_session, relationship
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.sql import and_, or_, not_, func

import csv
import matplotlib.pyplot as plt


In [ ]:
basepath = '/Users/eolson/WorkUBC/SWC/SQLAlchemy/'
dbname = 'fakeData'

## Create Database

In [ ]:
engine = create_engine('sqlite:///' + basepath + dbname + '.sqlite')

### Create Tables in Database
    tables are associated with Classes

In [ ]:
Base=declarative_base() # useful for creating your tables, defining their structure

In [ ]:
# We will use this first table as an intermediate step to import the flat spreadsheet
class LoadDataTBL(Base):
    __table__ = Table('LoadDataTBL', Base.metadata, 
                Column('ID', Integer, primary_key=True),
                Column('Station', Integer),
                Column('Lat', Numeric),
                Column('Lon', Numeric),
                Column('Date', String),
                Column('Depth', Numeric),
                Column('Temperature', Numeric),
                Column('Salinity', Numeric))

# Station Table
class StationTBL(Base):
    __table__=Table('StationTBL', Base.metadata,
                Column('ID', Integer, primary_key=True),
                Column('Lat', Numeric),
                Column('Lon', Numeric),
                Column('Date', String))

# Profile Table
class ProfileTBL(Base):
    __table__=Table('ProfileTBL', Base.metadata,
                Column('ID', Integer, primary_key=True),
                Column('StationTBLID', Integer, ForeignKey('StationTBL.ID')),
                Column('Depth', Numeric),
                Column('Temperature', Numeric),
                Column('Salinity', Numeric))

Base.metadata.create_all(engine)

### create session: this is how you interact with the database

In [ ]:
session = create_session(bind = engine, autocommit = False, autoflush = True)

### load data from csv file into 'LoadDataTBL':

In [ ]:
f=open(basepath+'MadeUpData.csv','r')
cf = csv.DictReader(f, delimiter=',')
i=0
for row in cf:
    i+=1
    if i<3: print(row) # print first two rows
    session.execute(LoadDataTBL.__table__.insert().values(**row))
f.close()

### Commit changes to database

In [ ]:
session.commit()

### Query database to return ID, Station, Depth, Temperature, and Salinity Columns from LoadDataTBL. We will insert  these in ProfileTBL

In [ ]:
# query LoadDataTBL to return columns that belong in ProfileTBL
# ID, Station, Depth, Temperature Salinity
q0=session.query(LoadDataTBL.ID,LoadDataTBL.Station,LoadDataTBL.Depth,LoadDataTBL.Temperature,
                LoadDataTBL.Salinity)

In [ ]:
for row in q0.all():
    print(*row)

In [ ]:
for row in q0.all():
    idict={}
    idict['ID']=row[0]
    idict['StationTBLID']=row[1]
    idict['Depth']=row[2]
    idict['Temperature']=row[3]
    idict['Salinity']=row[4]
    # enter in new record in Profile table:
    session.execute(ProfileTBL.__table__.insert().values(**idict))

In [ ]:
session.commit()

### Query database to return unique Station, Lat, Lon, and Depth Columns from LoadDataTBL. We will insert  these in ProfileTBL.

In [ ]:
q1=session.query(LoadDataTBL.Station,LoadDataTBL.Lat,LoadDataTBL.Lon,
                 LoadDataTBL.Date).group_by(LoadDataTBL.Station)

In [ ]:
for row in q1.all():
    print(*row)

In [ ]:
for Station, Lat, Lon, Date in q1.all():
    idict={}
    idict['ID']=Station
    idict['Lat']=Lat
    idict['Lon']=Lon
    idict['Date']=Date
    # enter in new record in Profile table:
    session.execute(StationTBL.__table__.insert().values(**idict))

In [ ]:
session.commit()

In [ ]:
session.close()

### Delete (drop) LoadDataTBL from database

In [ ]:
LoadDataTBL.__table__.drop(engine)

### close database connection

In [ ]:
engine.dispose()

## Reopen database and map tables to classes

In [ ]:
# automatically reflect database structure:
Base = automap_base()
engine = create_engine('sqlite:///' + basepath + dbname + '.sqlite', echo = False)
# reflect the tables:
Base.prepare(engine, reflect=True)
# mapped classes have been created

In [ ]:
# assign table classes to short variables for convenience:
StationTBL=Base.classes.StationTBL
ProfileTBL=Base.classes.ProfileTBL

In [ ]:
session = create_session(bind = engine, autocommit = False, autoflush = True)

## QUERIES
### Query to return all Stations with Latitude > 48.4

In [ ]:
q2=session.query(StationTBL.ID,StationTBL.Lat).filter(StationTBL.Lat>48.4)

In [ ]:
for row in q2.all():
    print(row)

### Query to return all Temperature, Salinity where Salinity < 33

In [ ]:
q3=session.query(ProfileTBL.Temperature,ProfileTBL.Salinity).filter(ProfileTBL.Salinity<33)

In [ ]:
for row in q3.all():
    print(row)

### --> Write a query to return all data from ProfileTBL where depth < 30

## How do we write a query that combines information from both tables?
### What happens if we jsut ask for information from both tables?

In [ ]:
q4=session.query(StationTBL.Lat,StationTBL.Lon,ProfileTBL.Temperature).\
        filter(ProfileTBL.Salinity<33)

In [ ]:
for row in q4:
    print(row)

### We need to join the tables using keys

In [ ]:
q4=session.query(StationTBL.Lat,StationTBL.Lon,ProfileTBL.Salinity).\
        select_from(ProfileTBL).join(StationTBL,StationTBL.ID==ProfileTBL.StationTBLID).\
        filter(ProfileTBL.Salinity<33)

In [ ]:
for row in q4:
    print(row)

### --> Write a query to return Depth, Temperature where Lat > 48.4

## Some built in functions are available to us. For instance, return the maximum salinity in the profile table:

In [ ]:
q5=session.query(func.max(ProfileTBL.Salinity))

In [ ]:
print(q5.one())

### These functions can also be applied over grouped sections of data

In [ ]:
q6=session.query(StationTBL.ID,func.avg(ProfileTBL.Temperature),func.count()).select_from(
        ProfileTBL).join(StationTBL,StationTBL.ID==ProfileTBL.StationTBLID).\
        group_by(StationTBL.ID)
for row in q6:
    print(row)

### --> Write a query that returns each Salinity value present in ProfileTBL along with the number of times it appears

## What if you want to use a column more than once in your query? For instance:
### Return ID, Temperature and Salinity pairs from records where both the depth and temperature are equal

In [ ]:
qP1=session.query(ProfileTBL).subquery()
qP2=session.query(ProfileTBL).subquery()

In [ ]:
qMatch=session.query(qP1.c.ID,qP2.c.ID,qP1.c.Temperature,qP2.c.Temperature,
                     qP1.c.Salinity,qP2.c.Salinity).filter(
        qP1.c.ID!=qP2.c.ID,qP1.c.Temperature==qP2.c.Temperature,qP1.c.Depth==qP2.c.Depth)

In [ ]:
for row in qMatch.all():
    print(*row)

### Put everything together:
### --> Write a query to return Latitudes and Longitudes where salinities at more than one depth were less than 29

In [ ]:
q0=session.query(ProfileTBL.StationTBLID,func.count().label('N')).filter(ProfileTBL.Salinity<29).\
            group_by(ProfileTBL.StationTBLID)

In [ ]:
for row in q0.all():
    print(row)

In [ ]:
qsub=q0.subquery()

In [ ]:
q=session.query(StationTBL.ID,StationTBL.Lat,StationTBL.Lon).\
        select_from(StationTBL).join(qsub,qsub.c.StationTBLID==StationTBL.ID).\
        filter(qsub.c.N>1)

In [ ]:
for row in q.all():
    print(row)

In [ ]:
session.close()
engine.dispose()